In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Define the User-Item Interaction Dataset
class UserItemDataset(Dataset):
    def __init__(self, interactions):
        self.interactions = interactions

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        return self.interactions[idx]

# Define the Collaborative Filtering Model
class CollaborativeFiltering(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CollaborativeFiltering, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim)

    def forward(self, user, item):
        user_embedding = self.user_embeddings(user)
        item_embedding = self.item_embeddings(item)
        dot_product = torch.sum(user_embedding * item_embedding, dim=1)
        return dot_product

# Prepare the data
interactions = [
    (0, 0), (0, 1), (0, 3),
    (1, 1), (1, 2),
    (2, 0), (2, 2), (2, 3),
    (3, 1), (3, 3)
]
num_users = 4
num_items = 4
embedding_dim = 8

dataset = UserItemDataset(interactions)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# Initialize the model, loss function, and optimizer
model = CollaborativeFiltering(num_users, num_items, embedding_dim)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    total_loss = 0
    for user, item in dataloader:
        optimizer.zero_grad()
        predictions = model(user, item)
        loss = loss_fn(predictions, torch.ones_like(predictions))  # Target values are all 1
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}")

# Make predictions for a specific user
test_user = torch.tensor([0, 0, 0, 1])
test_item = torch.tensor([1, 2, 3, 3])
predictions = model(test_user, test_item)
recommended_items = torch.argmax(predictions)

print(f"User: {test_user.tolist()}")
print(f"Recommended Items: {recommended_items.tolist()}")


Epoch 1/10, Loss: 16.2503
Epoch 2/10, Loss: 14.2346
Epoch 3/10, Loss: 12.4012
Epoch 4/10, Loss: 11.2645
Epoch 5/10, Loss: 9.7692
Epoch 6/10, Loss: 8.5969
Epoch 7/10, Loss: 6.9695
Epoch 8/10, Loss: 6.9970
Epoch 9/10, Loss: 5.6598
Epoch 10/10, Loss: 5.3699
User: [0, 0, 0, 1]
Recommended Items: 0


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from sklearn.neighbors import NearestNeighbors

# Define the User-Item Interaction Dataset
class UserItemDataset(Dataset):
    def __init__(self, interactions):
        self.interactions = interactions

    def __len__(self):
        return len(self.interactions)

    def __getitem__(self, idx):
        return self.interactions[idx]

# Prepare the data
interactions = [
    (0, 0), (0, 1), (0, 3),
    (1, 1), (1, 2),
    (2, 0), (2, 2), (2, 3),
    (3, 1), (3, 3)
]
num_users = 4
num_items = 4

dataset = UserItemDataset(interactions)

# Collaborative Filtering Model
class CollaborativeFiltering(nn.Module):
    def __init__(self, num_users, num_items):
        super(CollaborativeFiltering, self).__init__()
        self.num_users = num_users
        self.num_items = num_items

    def forward(self, user, item):
        pass  # Not used for KNN

# Fit the KNN model
X_train = torch.tensor(interactions)
knn_model = NearestNeighbors(metric='cosine')
knn_model.fit(X_train)

# Make predictions for a specific user
test_user = torch.tensor([0, 0, 0, 1]).reshape(1, -1)
_, indices = knn_model.kneighbors(test_user)
recommended_items = indices.squeeze()

print(f"User: {test_user.tolist()}")
print(f"Recommended Items (KNN): {recommended_items.tolist()}")


ValueError: X has 4 features, but NearestNeighbors is expecting 2 features as input.